In [ ]:
# %%

import torch
from torch.func import jacrev, functional_call
import numpy as np
from pyDOE import lhs
from lm_train.network import DNN
from lm_train.training_module import training_LM

In [ ]:
# %%

def model_u(data, params):
    return functional_call(model, params, (data, )).squeeze()


def exact(data):
    """The exact solution"""
    x = data[..., 0]
    y = data[..., 1]
    return torch.sin(np.pi * x) * torch.sin(np.pi * y)


def force(data):
    """The force function."""
    x = data[..., 0]
    y = data[..., 1]
    return -2 * np.pi**2 * torch.sin(np.pi * x) * torch.sin(np.pi * y)


def generate2dRandomPoints(domainPoints, boundaryPoints, start=0., end=1.):
    """Generate 2d random points given start and end point."""
    data_d = torch.tensor(lhs(2, domainPoints))
    data_b = torch.empty((4 * int(boundaryPoints / 4), 2))
    n_edge = int(boundaryPoints / 4)
    sample_b = torch.tensor(lhs(1, boundaryPoints)).flatten()
    ones = torch.ones(n_edge)
    data_b[0:n_edge] = torch.column_stack((sample_b[0:n_edge], 0 * ones))
    data_b[n_edge:2 * n_edge] = torch.column_stack(
        (0 * ones, sample_b[n_edge:2 * n_edge]))
    data_b[2 * n_edge:3 * n_edge] = torch.column_stack(
        (ones, sample_b[2 * n_edge:3 * n_edge]))
    data_b[3 * n_edge:4 * n_edge] = torch.column_stack(
        (sample_b[3 * n_edge:], ones))

    # rescale the data
    data_d = (end - start) * data_d + start
    data_b = (end - start) * data_b + start
    return data_d, data_b

# %% [markdown]

 $$\Delta u = f \text{ in } D$$
 $$ u = g \text{ on } \partial D$$

In [ ]:
# %%

def loss_residual(params, *args, **kwargs):
    "laplace u = f"
    data, force_value, = args
    grad_f = (jacrev(jacrev(model_u)))(data, params).squeeze()
    laplacian = torch.sum(torch.diagonal(grad_f))
    assert laplacian.shape == force_value.shape, 'The shape of laplacian and force_value should match'
    loss_d = laplacian - force_value
    return loss_d


def loss_target(params, *args, **kwargs):
    "General target loss"
    data, target, = args
    output = model_u(data, params)
    assert output.shape == target.shape, 'The shape of output and target should match'
    loss_b = output - target
    return loss_b

In [ ]:
# %%

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
layers = [2, 100, 1]
n_points_d = 1000
n_points_b = 100
n_epoch = 1000
model = DNN(layers).to(device)
params = dict(model.named_parameters())
data_d, data_b = generate2dRandomPoints(n_points_d, n_points_b)
data_d = data_d.to(device)
data_b = data_b.to(device)
force_value = force(data_d).to(device)
target = exact(data_b).to(device)

In [ ]:
# %%

losses = [loss_residual, loss_target]
inputs = [[
    data_d,
    force_value,
], [
    data_b,
    target,
]]
kwargs = [{} for _ in range(len(losses))]
args = tuple(zip(losses, inputs, kwargs))

In [ ]:
# %%

params, lossval_all, loss_running, lossval_test = training_LM(params,
                                                              device,
                                                              args,
                                                              steps=n_epoch)

Step: 100. loss: 4.8112e-04. mu: 3.6024e-13.
Step: 200. loss: 1.7479e-11. mu: 5.6936e-15.
Step: 300. loss: 1.1541e-12. mu: 3.2396e-15.
Step: 400. loss: 4.9759e-13. mu: 1.8433e-15.
Step: 500. loss: 3.8910e-13. mu: 6.2928e-15.
Step: 600. loss: 1.2358e-13. mu: 3.5805e-15.
Step: 700. loss: 9.0057e-14. mu: 2.0373e-15.
Step: 800. loss: 5.8094e-14. mu: 6.9550e-15.
Step: 900. loss: 4.0326e-14. mu: 3.9573e-15.
Step: 1000. loss: 1.3307e-14. mu: 2.2516e-15.
training time: 6.309163331985474 (s).


In [ ]:
# %%

# calculate the L_inf error
data_d_test, data_b_test = generate2dRandomPoints(int(1e6), int(1e4))
data_test = torch.row_stack((data_d_test, data_b_test)).to(device)
output = model_u(data_test, params)
target = exact(data_test)
error = torch.linalg.norm(output - target, float('inf'))
print(f'The L_inf error is: {error:.4e}')

The L_inf error is: 1.4253e-07
